# NLP analysis objectives
- switch to outscraper to extract full information of reviews ✓
- check if the place might have water army?
- extract the most related
- use sentiment analysis to analysis the score rather than using google score
- NLTK

# To Do:
- download a set of data with only reviews and rating or Aurther ✓
- Word2vec to show its true reviews
- sentiment analysis to measure its true value
## data preprocessing:
- stopword ✓
- lemmetize ✓
- porterstemmer (clean liked like liking into like) ✓
- embedding ✓
- train test split ✓
- word2vec after cleaning ✓
- onehotencoding y as well

# Tech issue
- maybe code a function to download necessary nlp trained model

In [1]:
import pandas as pd
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.tokenize import word_tokenize
import numpy as np
import re,string
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /home/aipen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/aipen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/aipen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/aipen/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
df=pd.read_csv('data_references/yelp.csv')
df=df[['text','stars']]

In [3]:
def clean_data(word):
    model= stopwords.words('english')
    trim_s=WordNetLemmatizer()
    stem=PorterStemmer()
    word = ''.join([i for i in str(word) if not i.isdigit()])
    word = word_tokenize(word.lower())
    word = [i for i in set(word) if i.isalpha() and i not in string.punctuation and i not in model]
    word = [trim_s.lemmatize(i) for i in word]
    word = [stem.stem(i) for i in word]
    
    return " ".join(word)

In [4]:
df['true_review']=np.vectorize(clean_data)(df.text)

In [5]:
X=df['true_review']
y=df[['stars']]

 # Hyper parameters for grid search:
## word2vec
- vector_size
- min_count
- window
## padding
- maxlen

In [ ]:
vector_size=[50]
window=[5,6,7,8]
maxlen=[10,20,30,40,50,60,70]

col_vec=[]
col_win=[]
col_max=[]

for a in vector_size:
    for b in window:
        for c in maxlen:
            col_vec.append(a)
            col_win.append(b)
            col_max.append(c)
        

df_test=pd.DataFrame({'vector':col_vec,'window':col_win,'maxlen':col_max})
hist_list=[]
for z in range(0,len(df_test)):
    new_y=[]
    for i in df.stars:
        if i ==5:
            new_y.append([0,0,0,0,1])
        elif i ==4:
            new_y.append([0,0,0,1,0])
        elif i ==3:
            new_y.append([0,0,1,0,0])
        elif i ==2:
            new_y.append([0,1,0,0,0])
        else:
            new_y.append([1,0,0,0,0])
    new_y=np.array(new_y)

    X_train, X_test, y_train, y_test = train_test_split(X, new_y, test_size=0.33, random_state=42)

    word2vec = Word2Vec(sentences=X_train, vector_size=df_test['vector'][z], min_count=1, window=df_test['window'][z])

    from tensorflow.keras.preprocessing.sequence import pad_sequences
    import numpy as np

    # Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
    def embed_sentence(word2vec, sentence):
        embedded_sentence = []
        for word in sentence:
            if word in word2vec.wv:
                embedded_sentence.append(word2vec.wv[word])

        return np.array(embedded_sentence)

    # Function that converts a list of sentences into a list of matrices
    def embedding(word2vec, sentences):
        embed = []

        for sentence in sentences:
            embedded_sentence = embed_sentence(word2vec, sentence)
            embed.append(embedded_sentence)

        return embed

    # Embed the training and test sentences
    X_train_embed = embedding(word2vec, X_train)
    X_test_embed = embedding(word2vec, X_test)

    X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=df_test['maxlen'][z])
    X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=df_test['maxlen'][z])

    from tensorflow.keras import Sequential
    from tensorflow.keras import layers

    def init_model():
        model = Sequential()
        model.add(layers.Masking())
        model.add(layers.LSTM(20, activation='tanh'))
        model.add(layers.Dense(15, activation='relu'))
        model.add(layers.Dense(10, activation='relu'))
        model.add(layers.Dense(5, activation='relu'))
        model.add(layers.Dense(5, activation='softmax'))

        model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])

        return model
    model=init_model()

    from tensorflow.keras.callbacks import EarlyStopping

    es = EarlyStopping(patience=10, restore_best_weights=True)

    history=model.fit(X_train_pad, y_train, 
              batch_size = 64,
              epochs=500,
              callbacks=[es],
              validation_data=(X_test_pad,y_test),
              verbose=0
             )  
    hist_list.append(history.history['val_accuracy'][-1])
    print(f'finished {z+1} out of {len(df_test)} with {int(((z+1)/len(df_test))*100)}% progress')

finished 1 out of 28 with 3% progress
finished 2 out of 28 with 7% progress
finished 3 out of 28 with 10% progress
finished 4 out of 28 with 14% progress
finished 5 out of 28 with 17% progress
finished 6 out of 28 with 21% progress
finished 7 out of 28 with 25% progress
finished 8 out of 28 with 28% progress
finished 9 out of 28 with 32% progress
finished 10 out of 28 with 35% progress
finished 11 out of 28 with 39% progress
finished 12 out of 28 with 42% progress
finished 13 out of 28 with 46% progress


In [ ]:
df_test['val_acc']=hist_list

In [ ]:
df_test

In [12]:
'''
!git add NLP.ipynb
!git commit -m 'create table for grid search'
!git push origin master
'''

"\n!git add NLP.ipynb\n!git commit -m 'try 500 epoch'\n!git push origin master\n"